In [22]:

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Load the dataset
df = pd.read_csv('TEST.csv')

In [23]:
df

,windspeed,rainfall,area,yield,percentageofarea,darea,yielda,fgprice,dctrice
0,0,1.535972,2870.00,3.37,0.247070,709.091324,0.832626398,20.79,11135504
1,0,2.141071,491.00,4.36,0.244428,120.014173,1.065706303,20.79,2412279.37
2,7.116629346,5.809375,1330.88,4.42,0.312983,416.543187,1.383386094,20.79,10868283.84
3,67.08946747,3.908854,5767.00,4.65,0.567852,3274.800267,2.640510013,20.79,163091064.2
4,0,0.805469,4710.00,4.14,0.130472,614.521934,0.540153038,20.79,6260539.25
...,...,...,...,...,...,...,...,...,...
21016,0,0.522989,2936.00,0,0.264956,777.910078,0,14.32,0
21017,0,0.110000,156.00,2.85,0.101081,15.768650,0.288081111,14.32,59014.04
21018,10.51271508,0.225000,6376.00,6.88,0.131057,835.618326,0.901670967,14.32,9788183.48
21019,2.823089172,0.379861,2065.00,3.14,0.115129,237.741383,0.361505057,14.32,1116516.69


In [24]:
# Split the dataset into features and target variable
X = df.drop(['dctrice','darea','percentageofarea','yielda','fgprice'], axis=1)
y = df.drop(['darea','percentageofarea','yielda','windspeed','rainfall','area', 'yield', 'fgprice'], axis=1)


In [25]:
X.shape, y.shape

((21021, 4), (21021, 1))

In [26]:
# Training and Temporary Set (80-20 split)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Temporary set into a validation set and a test set using (50-50 split)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [27]:

X_train.shape, y_train.shape

((16816, 4), (16816, 1))

In [28]:
y_train.info()
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16816 entries, 13628 to 15795
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dctrice  16816 non-null  object
dtypes: object(1)
memory usage: 262.8+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16816 entries, 13628 to 15795
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   windspeed  16816 non-null  object 
 1   rainfall   16816 non-null  float64
 2   area       16816 non-null  float64
 3   yield      16705 non-null  object 
dtypes: float64(2), object(2)
memory usage: 656.9+ KB


In [29]:
X_test = pd.DataFrame(X_test).apply(pd.to_numeric, errors='coerce')
X_train = pd.DataFrame(X_train).apply(pd.to_numeric, errors='coerce')
y_train = pd.DataFrame(y_train).apply(pd.to_numeric, errors='coerce')
y_test = pd.DataFrame(y_test).apply(pd.to_numeric, errors='coerce')

In [30]:
X_train.shape, y_train.shape

((16816, 4), (16816, 1))

In [31]:
# Remove the rows with zero values from the training set and testing set
mask = X_train['windspeed'] != 0

# Apply the mask to X_train
X_train = X_train[mask]

# Apply the mask to y_train
y_train = y_train[mask]

In [32]:
X_train.shape, y_train.shape

((7867, 4), (7867, 1))

In [33]:
# Null Remover

nan_rows = X_train[X_train.isna().any(axis=1)].index
X_train = X_train.drop(nan_rows)
y_train = y_train.drop(nan_rows)

print(np.where(pd.isnull(X_train)))

(array([], dtype=int64), array([], dtype=int64))


In [34]:
X_train.shape,y_train.shape

((7837, 4), (7837, 1))

In [35]:
# mask = X_train['windspeed'] != 0


In [36]:
nan_rows = X_test[X_test.isna().any(axis=1)].index
X_test = X_test.drop(nan_rows)
y_test = y_test.drop(nan_rows)

In [37]:
# mask = X_test['windspeed'] != 0

# # Apply the mask to X_train
# X_test = X_test[mask]

# y_test = y_test[mask]


In [38]:
from sklearn.ensemble import StackingRegressor
from joblib import load, dump

# Load the models
svr_pipeline = load('SVR.joblib')
rf = load('RFR.joblib')

# Create a StackingRegressor
stacking_regressor = StackingRegressor(
    estimators=[('svr', svr_pipeline), ('rf', rf)]
)

# Fit the StackingRegressor to the training data
# Note: This step is necessary to train the meta-estimator
stacking_regressor.fit(X_train, y_train)

predictions = stacking_regressor.predict(X_test)

# Save the StackingRegressor
dump(stacking_regressor, 'stacking_regressor.joblib')

c:\Users\Asus\anaconda3\lib\site-packages\sklearn\ensemble\_stacking.py:758: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['stacking_regressor.joblib']

In [39]:
nan_rows = X_val[X_val.isna().any(axis=1)].index
X_val = X_val.drop(nan_rows)
y_val = y_val.drop(nan_rows)

In [40]:
from sklearn.metrics import mean_absolute_error


predictionsVal = stacking_regressor.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, predictionsVal)
print(f'Mean Squared Error: {mse}')

mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error (Test): {mae}')

r2 = stacking_regressor.score(X_test, y_test)
print(f'R-squared (Test): {r2}')

Mean Squared Error: 1.2814160351562565e+20
Mean Absolute Error (Test): 6452731576.031885
R-squared (Test): -201030.50689484985


In [41]:
print(['{:.2f}'.format(pred) for pred in predictionsVal])

['852606129.48', '3669319041.26', '3260693458.82', '5767395239.74', '5767395239.74', '5767395239.74', '5767395239.73', '5767395239.74', '4014982556.72', '4014982556.72', '4663493436.63', '4014982556.72', '-5573593831.33', '-58580066781.25', '-2694885975.62', '3462006340.55', '5767395239.73', '5767395239.72', '5767395239.72', '5767395239.70', '5407815345.40', '5767395239.74', '3538436139.52', '1885635836.20', '5767395239.74', '-46189081480.04', '4490554178.41', '4486238781.96', '5327519447.27', '5767395239.74', '5767395239.71', '5767395239.73', '-10564739592.69', '4630857739.67', '4569338081.00', '-6433792692.63', '5767395239.74', '-2608759621.63', '5767395239.71', '5767395239.71', '-1069843266.77', '5767395239.73', '3008212228.14', '3008212228.13', '5213131002.29', '4523876129.97', '5767395239.72', '852606129.48', '4466660400.68', '5767395239.72', '5767395239.74', '-6963507577.87', '5767395239.74', '5767395239.74', '5767395239.73', '5767395239.73', '5767395239.74', '5767395239.74', '40